In [4]:
import requests
import time
import re
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
url = 'https://www.kicktraq.com/archive/'

response = requests.get(url)

In [6]:
response.status_code

200

In [7]:
page = response.text

In [8]:
soup = BeautifulSoup(page,'lxml')

In [25]:
kickstarter_page = []
funding = []
campaign_dates = []

all_projects_on_page = soup.find_all(class_="project first odd closed")
all_projects_on_page = all_projects_on_page + soup.find_all(class_="project odd closed")
all_projects_on_page = all_projects_on_page + soup.find_all(class_="project closed")

for project in all_projects_on_page: 
    #projects.append(project)
    kickstarter_page.append(project.find('h2').find('a')['href'])
    funding.append(project.find(class_='project-details').find(text=re.compile('Funding: .\d* of .\d*.\d*')))
    campaign_dates.append(project.find(class_='project-details').find(text=re.compile('Dates: \w* \d*\w* \-\> \w* \d*\w* \(\d*\)')))

In [28]:
campaign_dates[0:5]

['\n\t\t\t\t\tCampaign Dates: November 26th -> January 25th (2015)',
 '\n\t\t\t\t\tCampaign Dates: November 24th -> January 23rd (2015)',
 '\n\t\t\t\t\tCampaign Dates: November 21st -> January 20th (2015)',
 '\n\t\t\t\t\tCampaign Dates: November 19th -> January 17th (2015)',
 '\n\t\t\t\t\tCampaign Dates: November 17th -> January 16th (2015)']

In [11]:
#projects[1]


[<h2><a href="/projects/1063028485/good-art-guide/">Good art guide (Canceled)</a></h2>, <h2><a href="/projects/2ucid/i-want-to-be-welcomed/">I Want To Be Welcomed...</a></h2>, <h2><a href="/projects/spiderwebcitytx/katlynn-maries-horror-and-other-makeup/">Katlynn Marie's Horror &amp; Other Makeup</a></h2>, <h2><a href="/projects/655647929/the-life-of-an-avid-gambler/">The Life Of An Avid Gambler (Canceled)</a></h2>, <h2><a href="/projects/mnassar/wwwparkinglocatorcomau/">www.Parkinglocator.com.au</a></h2>, <h2><a href="/projects/576040839/my-dead-friends-web-series/">My Dead Friends Web Series</a></h2>, <h2><a href="/projects/1934089488/not-enough-fish/">Not Enough Fish</a></h2>, <h2><a href="/projects/458506913/gift-of-designs/">Gift Of Designs</a></h2>, <h2><a href="/projects/timodwyer/hurt-fate-feature-film/">Hurt Fate Feature Film</a></h2>, <h2><a href="/projects/1988967617/the-riders-perspective/">The Rider's Perspective</a></h2>, <h2><a href="/projects/1110787116/jules-writes-a-b

In [ ]:
projects=pd.DataFrame(columns=['kickstarter_page','funding','funding_goal', 'campaign dates'])

In [ ]:

for page_num in range(1,8000):
    url = 'https://www.kicktraq.com/archive/?page='+str(page_num)
    
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,'lxml')
    
    
    
    
    
    time.sleep(11)
